# 词向量构建与表示

- [TF-IDF](#TF-IDF)
- [Word2Vec](#Word2Vec)

## TF-IDF

In [1]:
import pandas as pd
import numpy as np 
import jieba.posseg
import jieba.analyse
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
%load_ext watermark
%watermark -m -v -p numpy,pandas,jieba,sklearn,gensim

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
jieba 0.40
sklearn 0.21.2
gensim 3.7.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [3]:
def data_preprocess(text):
    l = []
    pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd'] # 定义选取的词性
    seg = jieba.posseg.cut(text) # 分词
    for i in seg:
        if i.word and i.flag in pos: # 词性筛选
            l.append(i.word)
    return l

In [4]:
def get_keywords_tfidf(data, top_k):
    '''打印 TOP-K TF-IDF 值
    
    Return：
        corpus 预处理后的语料
    '''
    id_list, title_list, abstract_list = data['index'], data['title'], data['abstract']
    corpus = []
    for index in range(len(id_list)):
        text = '%s。 %s' % (title_list[index], abstract_list[index])
        text = data_preprocess(text) # 文本预处理
        text = ' '.join(text) # 按 sklearn 要求用空格分隔
        corpus.append(text)
        
    # 构造词频矩阵    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    
    # 统计每个词的 tf-idf 权值
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(X)
    
    # 获取词袋模型里的关键词
    word = vectorizer.get_feature_names()
    
    # 获取 tf-idf 矩阵
    weight = tfidf.toarray()
    for i in range(len(weight)):
        print('---------这里输出第', i+1, '篇文本的词语 tf-idf--------')
        df = pd.DataFrame(weight[i], index = word, columns=['tfidf'])
        df = df.sort_values(by=['tfidf'], ascending=False)
        top_k_arr = np.array(df[:top_k].reset_index())
        for j in top_k_arr:
            print(j)
    return corpus

In [5]:
data_path = './data/summary_sample.csv' # 1103 条语料
data = pd.read_csv(data_path, names=['title','abstract'])
data = data.reset_index()
data = data[:10]
data.head()

,index,title,abstract
0,0,可穿戴技术十大设计原则,本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1...
1,1,经济学人：智能手机将成为“真正的个人电脑”,2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而...
2,2,雅虎宣布剥离阿里巴巴股份,雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打...
3,3,51信用卡管家，预计2015年放贷额度远超30亿,2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线...
4,4,如何选择正确的编程语言进行学习,目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言...


In [6]:
raw_data = get_keywords_tfidf(data, 10)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\mfx66\AppData\Local\Temp\jieba.cache
Loading model cost 0.903 seconds.
Prefix dict has been built successfully.


---------这里输出第 1 篇文本的词语 tf-idf--------
['原则' 0.4756440656369588]
['开始' 0.31709604375797257]
['穿戴' 0.31709604375797257]
['设计' 0.31709604375797257]
['能力' 0.15854802187898628]
['取代' 0.15854802187898628]
['注意' 0.15854802187898628]
['行业' 0.15854802187898628]
['机器' 0.15854802187898628]
['解决' 0.15854802187898628]
---------这里输出第 2 篇文本的词语 tf-idf--------
['智能手机' 0.49580532163083624]
['真正' 0.3305368810872242]
['个人电脑' 0.3305368810872242]
['成为' 0.28098654676649554]
['代表' 0.1652684405436121]
['宣称' 0.1652684405436121]
['展示' 0.1652684405436121]
['已经' 0.1652684405436121]
['贡献' 0.1652684405436121]
['触屏' 0.1652684405436121]
---------这里输出第 3 篇文本的词语 tf-idf--------
['雅虎' 0.511159223641019]
['剥离' 0.3407728157606793]
['第四季度' 0.17038640788033965]
['计划' 0.17038640788033965]
['分配' 0.17038640788033965]
['发稿' 0.17038640788033965]
['股份' 0.17038640788033965]
['股价' 0.17038640788033965]
['股东' 0.17038640788033965]
['上涨' 0.17038640788033965]
---------这里输出第 4 篇文本的词语 tf-idf--------
['放贷' 0.42591437652078784]
['信用卡' 0.4259

## Word2Vec

In [8]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [9]:
gensim_corpus = [i.split(' ') for i in raw_data]
gensim_corpus

[['可',
  '穿戴',
  '技术',
  '设计',
  '原则',
  '总结',
  '可',
  '穿戴',
  '产品',
  '设计',
  '原则',
  '原则',
  '同样',
  '也',
  '是',
  '笔者',
  '认为',
  '是',
  '行业',
  '最',
  '吸引',
  '人',
  '地方',
  '人们',
  '解决',
  '重复性',
  '问题',
  '人',
  '开始',
  '机器',
  '开始',
  '要',
  '引起',
  '注意',
  '刻意',
  '提升',
  '用户',
  '能力',
  '取代',
  '人'],
 ['经济学',
  '人',
  '智能手机',
  '将',
  '成为',
  '真正',
  '个人电脑',
  '人们',
  '展示',
  '宣称',
  '将',
  '会',
  '改变',
  '世界',
  '还有',
  '人',
  '认为',
  '代表',
  '触屏',
  '智能手机',
  '已经',
  '席卷',
  '全球',
  '角落',
  '智能手机',
  '将',
  '会',
  '成为',
  '真正',
  '个人电脑',
  '人类',
  '发展',
  '做出',
  '更',
  '大',
  '贡献'],
 ['雅虎',
  '宣布',
  '剥离',
  '股份',
  '雅虎',
  '发布',
  '第四季度',
  '财报',
  '推出',
  '免税',
  '方式',
  '剥离',
  '持有',
  '集团',
  '股权',
  '计划',
  '打算',
  '将',
  '价值',
  '约',
  '投资',
  '分配',
  '股东',
  '截止',
  '发稿',
  '雅虎',
  '股价',
  '上涨',
  '大约'],
 ['信用卡',
  '管家',
  '预计',
  '放贷',
  '额度',
  '远',
  '超',
  '信用卡',
  '管家',
  '信',
  '公司',
  '合作',
  '推出',
  '线',
  '信贷',
  '产品',
  '贷',
  '是',
  '纯',
  '操作',
  '信贷',
 

In [10]:
# sg = 1 skipgram, sg = 0 cbow
model =  Word2Vec(gensim_corpus, size=100, window=5, min_count=1, workers=4, sg=1, negative=5, ns_exponent=0.75)

In [12]:
model.wv.most_similar('笔者')

[('股份', 0.31312453746795654),
 ('选择', 0.284382164478302),
 ('发展', 0.28047311305999756),
 ('信息', 0.2589920163154602),
 ('产品', 0.250807523727417),
 ('原因', 0.22913417220115662),
 ('创新', 0.2276233285665512),
 ('光头', 0.21164679527282715),
 ('受众', 0.20610925555229187),
 ('不能', 0.204687237739563)]

In [ ]:
%%script false
model.save('./data/w2v_gensim')

------------

**作者：** Daniel Meng

**GitHub：** [LibertyDream](https://github.com/LibertyDream)

**博客：** [明月轩](https://libertydream.github.io/)